# Jednoduchý klasifikátor obrázků

Tento notebook vám ukáže, jak klasifikovat obrázky pomocí předem natrénované neuronové sítě.

**Co se naučíte:**
- Jak načíst a použít předem natrénovaný model
- Předzpracování obrázků
- Provádění predikcí na obrázcích
- Porozumění skóre důvěryhodnosti

**Použití:** Identifikace objektů na obrázcích (například "kočka", "pes", "auto" atd.)

---


## Krok 1: Importujte potřebné knihovny

Importujeme nástroje, které budeme potřebovat. Nemějte obavy, pokud zatím všemu nerozumíte!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Krok 2: Načtení předem natrénovaného modelu

Použijeme **MobileNetV2**, neuronovou síť, která byla již natrénována na milionech obrázků.

Tomu se říká **Transfer Learning** - využití modelu, který natrénoval někdo jiný!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Krok 3: Pomocné funkce

Vytvořme funkce pro načítání a přípravu obrázků pro náš model.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Krok 4: Testování na vzorových obrázcích

Zkusme klasifikovat některé obrázky z internetu!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klasifikujte každý obrázek


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Krok 5: Vyzkoušejte své vlastní obrázky!

Nahraďte níže uvedenou URL libovolnou URL obrázku, který chcete klasifikovat.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Co se právě stalo?

1. **Načetli jsme předtrénovaný model** - MobileNetV2 byl natrénován na milionech obrázků  
2. **Předzpracovali jsme obrázky** - Změnili jsme jejich velikost a formát pro model  
3. **Model provedl predikce** - Vygeneroval pravděpodobnosti pro 1000 kategorií objektů  
4. **Dekódovali jsme výsledky** - Převedli čísla na čitelné popisky  

### Porozumění skóre jistoty

- **90-100 %**: Velmi jisté (téměř určitě správné)  
- **70-90 %**: Jisté (pravděpodobně správné)  
- **50-70 %**: Částečně jisté (může být správné)  
- **Pod 50 %**: Málo jisté (nejisté)  

### Proč mohou být predikce chybné?

- **Neobvyklý úhel nebo osvětlení** - Model byl trénován na běžných fotografiích  
- **Více objektů** - Model očekává jeden hlavní objekt  
- **Vzácné objekty** - Model zná pouze 1000 kategorií  
- **Nízká kvalita obrázku** - Rozmazané nebo pixelované obrázky jsou obtížnější  


## 🚀 Další kroky

1. **Vyzkoušejte různé obrázky:**
   - Najděte obrázky na [Unsplash](https://unsplash.com)
   - Klikněte pravým tlačítkem → „Kopírovat adresu obrázku“ pro získání URL

2. **Experimentujte:**
   - Co se stane s abstraktním uměním?
   - Dokáže rozpoznat objekty z různých úhlů?
   - Jak si poradí s více objekty?

3. **Zjistěte více:**
   - Prozkoumejte [lekce o počítačovém vidění](../lessons/4-ComputerVision/README.md)
   - Naučte se trénovat vlastní klasifikátor obrázků
   - Pochopte, jak fungují CNNs (Konvoluční neuronové sítě)

---

## 🎉 Gratulujeme!

Právě jste vytvořili klasifikátor obrázků pomocí špičkové neuronové sítě!

Tato stejná technika pohání:
- Google Photos (organizace vašich fotografií)
- Autonomní auta (rozpoznávání objektů)
- Lékařskou diagnostiku (analýza rentgenových snímků)
- Kontrolu kvality (detekce vad)

Pokračujte v objevování a učení! 🚀



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby AI pro překlady [Co-op Translator](https://github.com/Azure/co-op-translator). I když se snažíme o přesnost, mějte prosím na paměti, že automatizované překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho původním jazyce by měl být považován za autoritativní zdroj. Pro důležité informace doporučujeme profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
